In [ ]:
from utils.tensorrt import InferenceCore

In [ ]:
from pycuda import autoinit
import numpy as np

In [ ]:
model = InferenceCore(
    "models/retinanet_540p.onnx",
    "models/retinanet_540p.engine",
    fp16_mode=True,
)
anchors = np.load("models/retinanet_540p_anchors.npy", allow_pickle=True)

In [ ]:
from PIL import Image
import numpy as np
img = Image.open("images/street.jpg")
img = img.resize((960, 540))
img = np.array(img)
img = np.rollaxis(img, 2, 0)
img = img[np.newaxis]

In [ ]:
img.shape

In [ ]:
outputs = model([img])

In [ ]:
for out in outputs:
    print(out.shape)

In [ ]:
from utils.postprocess import postprocess

In [ ]:
results = postprocess(outputs, anchors, [1.0, 1.0, 1.0, 1.0], 0.5, 0.2, 1000, 50, True)

In [ ]:
results.shape

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patches
img = Image.open("images/street.jpg")
img = img.resize((960, 540))
img = np.array(img)
fig, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(img)
for box in results[0]:
    x, y, w, h, label, confidence = box
    if confidence > 0.5:
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
plt.show()

In [ ]:
from time import time
now = time()
for _ in range(10):
    outputs = model([np.zeros((1, 3, 540, 960))])
    results = postprocess(outputs, anchors, [1.0, 1.0, 1.0, 1.0], 0.5, 0.2, 1000, 50, True)
dur = (time() - now) / 10
print("dur =", dur)
print("fps =", 1 / dur)

In [ ]:
from time import time
now = time()
for _ in range(10):
    outputs = model([np.zeros((1, 3, 540, 960))])
dur = (time() - now) / 10
print("dur =", dur)
print("fps =", 1 / dur)